In [1]:
import torch
import torchtext
import torch.functional as F
import torch.nn as nn
import spacy
import pandas as pd
import numpy
import random
import time
from torchtext.legacy.data import LabelField, Field, TabularDataset, BucketIterator
SEED=42
torch.manual_seed(SEED)

In [2]:
# path = r"C:\Users\win10\Downloads\Datasets\HS Dataset\labeled_data.csv"
# df = pd.read_csv(path)

# subset = {'tweet':[],'label':[]}
# hcount,ocount, ncount = 0,0,0
# for tweet, label in zip(df['tweet'],df['class']):
#     if(label == 0 and hcount<=1400):
#         hcount += 1
#         subset['tweet'].append(tweet)
#         subset['label'].append(label)
#     elif(label == 1 and ocount<=1400):
#         ocount += 1
#         subset['tweet'].append(tweet)
#         subset['label'].append(label)
#     elif(label == 2 and ncount<=1400):
#         ncount+= 1
#         subset['tweet'].append(tweet)
#         subset['label'].append(label)

# dataset = pd.DataFrame(data = subset, columns = ['tweet','label'])
# dataset.head(10)

# dataset = dataset.sample(frac=1).reset_index(drop=True)

# dataset.head(10)

# dataset.to_csv('dataset.csv')

In [3]:
tweet = Field(sequential=True, use_vocab=True, tokenize = 'spacy',tokenizer_language = 'en_core_web_sm', include_lengths=True, lower=True)
class_label = LabelField()

In [4]:
fields = {'tweet': ('text',tweet), 'label':('c', class_label)}

In [5]:
path = r"dataset.csv"
data= TabularDataset(path=path, format = 'CSV', fields = fields)

In [6]:
i = random.randint(0,4200)
data.examples[i].text, data.examples[i].c

(['"',
  '@dgotbricks',
  ':',
  'what',
  'happen',
  'to',
  'them',
  'vixen',
  'ent',
  'bitches',
  '"',
  'they',
  'got',
  'ran',
  'and',
  'threw',
  'to',
  'the',
  'side',
  'like',
  'a',
  'foothill',
  'bitch'],
 '1')

In [7]:
state = random.getstate()
train_data, test_data = data.split(split_ratio=0.85, random_state=state)

In [8]:
train_data[654].text

['@devilgrimz',
 'ironic',
 'that',
 'he',
 "'s",
 'a',
 'trashcan',
 'because',
 'he',
 'holds',
 'trash',
 'everytime',
 'you',
 'two',
 'cuddle',
 '!',
 'burn',
 '.']

In [9]:
tweet.build_vocab(train_data, max_size = 10000, min_freq = 15)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [10]:
class_label.build_vocab(train_data)
print(class_label.vocab.stoi)

defaultdict(None, {'1': 0, '0': 1, '2': 2})


In [11]:
train_iterator, test_iterator = BucketIterator.splits(
    (train_data,test_data),
    batch_size = 8,
    sort = False,
    device = 'cuda')

In [98]:
# for batch in train_iterator:
#     print(batch.text)
#     break

In [13]:
tweet.build_vocab(train_data)
tweet.vocab.load_vectors('glove.6B.300d')
embedding = tweet.vocab.vectors.to(device)

In [14]:
batch = next(iter(train_iterator))

In [97]:
# for i,batch in enumerate(train_iterator):
#     c = batch.c
#     a,b = batch.text
#     print(i,a,b,c)
#     break
# #     if(i==2):
# #         break

In [16]:
embedding.shape

torch.Size([11981, 300])

In [17]:
class LSTMClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, label_size, batch_size, embedding_weights, bidirectional = False):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.word_embeddings = nn.Embedding.from_pretrained(embedding_weights)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,
                            bidirectional = bidirectional,batch_first=True)
        if bidirectional:
            self.fc = nn.Linear(hidden_dim*2, label_size)
        else:
            self.fc = nn.Linear(hidden_dim, label_size)
#         self.act = nn.Sigmoid()
 
    def forward(self, sentence, src_len, train = True):
        embeds = self.word_embeddings(sentence)
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embeds, src_len.cpu(), enforce_sorted=False)
        packed_outputs, (hidden,cell) = self.lstm(packed_embedded)
#         hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
#         print(hidden.shape)
        dense_outputs = self.fc(hidden)
        outputs=dense_outputs
        return outputs

In [18]:
BATCH_SIZE = 32
nlabel = 3
hidden_dim = 25

model = LSTMClassifier(embedding_dim=embedding.shape[1],hidden_dim=hidden_dim,label_size=nlabel, batch_size=BATCH_SIZE, embedding_weights=embedding)
model = model.to(device)
 
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
loss_function = nn.CrossEntropyLoss()
 
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch
    """
    top_pred = preds.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [19]:
from tqdm import tqdm
epochs=15
for epoch in range(epochs):
    time.sleep(1)
    total_loss = 0.0
    total_acc=0.0
    for i, batch in enumerate(tqdm(train_iterator)):
        (feature, batch_length), label = batch.text, batch.c

        optimizer.zero_grad()
        output = model(feature, batch_length).squeeze()
        loss = loss_function(output, label)
        acc=categorical_accuracy(output,label)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_acc+=acc.item() 

    print(f"loss on epoch {epoch+1} = {total_loss/len(train_iterator)}")
    print(f"accuracy on epoch {epoch+1} = {total_acc/len(train_iterator)}")

100%|███████████████████████████████████████████████████████████████████████████████| 447/447 [00:03<00:00, 148.75it/s]


loss on epoch 1 = 0.7032023182584669
accuracy on epoch 1 = 0.7095637584159304


100%|███████████████████████████████████████████████████████████████████████████████| 447/447 [00:02<00:00, 180.63it/s]


loss on epoch 2 = 0.3742556309773351
accuracy on epoch 2 = 0.866442953046803


100%|███████████████████████████████████████████████████████████████████████████████| 447/447 [00:02<00:00, 192.44it/s]


...

loss on epoch 13 = 0.04538241916465556
accuracy on epoch 13 = 0.9872483221743198


100%|███████████████████████████████████████████████████████████████████████████████| 447/447 [00:02<00:00, 177.73it/s]


loss on epoch 14 = 0.05146512244171744
accuracy on epoch 14 = 0.9885346756152126


100%|███████████████████████████████████████████████████████████████████████████████| 447/447 [00:02<00:00, 180.74it/s]

loss on epoch 15 = 0.030187209481016632
accuracy on epoch 15 = 0.9921700223713646


In [20]:
m = nn.LogSoftmax(dim = 1)
_, idx = torch.max(m(output[0].reshape(1,3)), dim=1)
idx

tensor([0], device='cuda:0')

In [21]:
output

tensor([[ 4.8745, -0.4071, -4.6319],
        [ 6.1766, -0.7928, -5.5313],
        [-1.7001, -2.7845,  3.3825],
        [ 6.3392, -0.4266, -5.9746],
        [ 2.4014,  3.6090, -7.6764],
        [-3.7703, -2.5575,  4.2581],
        [-2.6498,  4.3665, -2.7020],
        [ 3.7369, -1.1688, -2.7710]], device='cuda:0',
       grad_fn=<SqueezeBackward0>)

In [22]:
output.argmax(dim=1)

tensor([0, 0, 2, 0, 1, 2, 1, 0], device='cuda:0')

In [23]:
batch_length.shape

torch.Size([8])

In [24]:
label

tensor([0, 0, 2, 0, 1, 2, 1, 0], device='cuda:0')

In [25]:
loss_function(output,label)

tensor(0.0361, device='cuda:0', grad_fn=<NllLossBackward>)

In [74]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_class(model, sentence, min_len = 4):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [tweet.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    preds = model(tensor, torch.tensor(tensor.shape[0], dtype = torch.int64).unsqueeze(0))
#     _, idx = torch.max(m(output[0].reshape(1,3)), dim=1)
    max_preds = preds[0].argmax(dim = 1)
#     print(max_preds)
    return max_preds

In [75]:
class_label.vocab.stoi

defaultdict(None, {'1': 0, '0': 1, '2': 2})

In [88]:
i = random.randint(0,629)
sentence = test_data[i].text

sen = ""
for word in sentence:
    sen = sen +" " + word

target_class = test_data[i].c

pred_class = predict_class(model, sen).item()
print(pred_class)
itol = {'0': 'Hate Speech', '1': 'Offensive', '2': 'Neither'}
print(str(i)+' '+sen)
print(f'Predicted class is: {pred_class} = {itol[class_label.vocab.itos[pred_class]]}. Target is: {itol[target_class]}')

0
534  " i txt my old bitch my new bitch pussy wetter "
Predicted class is: 0 = Offensive. Target is: Offensive


In [96]:
test_sen = 'Model is a fag'
pred_class = predict_class(model, test_sen).item()
itol = {'0': 'Hate Speech', '1': 'Offensive', '2': 'Neither'}
print(test_sen)
print(f'Predicted class is: {pred_class} = {itol[class_label.vocab.itos[pred_class]]}.')

Model is a fag
Predicted class is: 1 = Hate Speech.


In [45]:
from tqdm import tqdm
total_acc=0.0
m = nn.LogSoftmax(dim = 1)
for i, batch in enumerate(tqdm(test_iterator)):
    (feature, batch_length), label = batch.text, batch.c

    optimizer.zero_grad()
    output = model(feature, batch_length).squeeze()
    acc=categorical_accuracy(output,label)
    total_acc+=acc.item() 
print(f"accuracy = {total_acc/len(test_iterator)}")

 30%|████████████████████████▌                                                        | 24/79 [00:00<00:00, 237.55it/s]

tensor([2, 2, 2, 2, 0, 0, 2, 0], device='cuda:0')
tensor([0, 2, 0, 0, 2, 2, 2, 0], device='cuda:0')
tensor([1, 2, 2, 0, 1, 1, 2, 0], device='cuda:0')
tensor([1, 2, 0, 0, 2, 0, 0, 1], device='cuda:0')
tensor([2, 2, 2, 2, 0, 0, 1, 0], device='cuda:0')
tensor([0, 0, 1, 1, 1, 2, 2, 1], device='cuda:0')
tensor([0, 0, 1, 1, 0, 0, 0, 1], device='cuda:0')
tensor([2, 2, 2, 2, 2, 2, 0, 0], device='cuda:0')
tensor([2, 0, 0, 1, 1, 0, 2, 1], device='cuda:0')
tensor([0, 1, 1, 2, 0, 2, 1, 1], device='cuda:0')
tensor([1, 1, 1, 2, 2, 1, 0, 0], device='cuda:0')
tensor([1, 0, 2, 2, 0, 2, 0, 0], device='cuda:0')
tensor([1, 1, 0, 0, 0, 0, 1, 2], device='cuda:0')
tensor([2, 1, 0, 0, 2, 0, 0, 0], device='cuda:0')
tensor([1, 2, 0, 2, 2, 1, 0, 2], device='cuda:0')
tensor([1, 2, 2, 0, 2, 0, 1, 2], device='cuda:0')
tensor([1, 1, 2, 0, 2, 0, 1, 2], device='cuda:0')
tensor([0, 2, 0, 2, 0, 0, 0, 2], device='cuda:0')
tensor([1, 1, 2, 1, 1, 2, 1, 0], device='cuda:0')
tensor([1, 1, 0, 1, 0, 0, 1, 0], device='cuda:0')


100%|█████████████████████████████████████████████████████████████████████████████████| 79/79 [00:00<00:00, 283.16it/s]

tensor([2, 2, 0, 0, 1, 0, 2, 0], device='cuda:0')
tensor([2, 1, 0, 2, 2, 1, 1, 2], device='cuda:0')
tensor([0, 0, 1, 1, 2, 0, 2, 2], device='cuda:0')
tensor([0, 0, 2, 0, 1, 0, 0, 1], device='cuda:0')
tensor([2, 0, 0, 1, 1, 1, 0, 1], device='cuda:0')
tensor([1, 0, 2, 0, 2, 1, 0, 2], device='cuda:0')
tensor([1, 1, 1, 2, 1, 1, 0, 2], device='cuda:0')
tensor([1, 2, 1, 2, 1, 2, 1, 1], device='cuda:0')
tensor([2, 1, 1, 2, 1, 2, 0, 1], device='cuda:0')
tensor([1, 2, 2, 0, 2, 2, 2, 0], device='cuda:0')
tensor([1, 2, 2, 2, 1, 0, 2, 1], device='cuda:0')
tensor([1, 2, 1, 0, 1, 2, 0, 0], device='cuda:0')
tensor([2, 0, 2, 1, 1, 2, 0, 2], device='cuda:0')
tensor([0, 1, 0, 1, 2, 1, 1, 1], device='cuda:0')
tensor([0, 2, 2, 0, 1, 0, 0, 2], device='cuda:0')
tensor([2, 0, 1, 0, 0, 2, 0, 0], device='cuda:0')
tensor([2, 2, 0, 1, 1, 2, 1, 0], device='cuda:0')
tensor([0, 2, 1, 2, 0, 1, 0, 0], device='cuda:0')
tensor([1, 2, 0, 1, 2, 2, 2, 2], device='cuda:0')
tensor([2, 2, 1, 0, 0, 0, 2, 1], device='cuda:0')


In [31]:
#Get count of examples of each class from dataset
# h, o, n = 0,0,0
# for example in train_data:
#     if(int(example.c)==0):
#         h += 1
#     elif(int(example.c)==1):
#         o += 1
#     elif(int(example.c)==2):
#         n += 1

# h,o,n